![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/header_workshop.png)

# Generate Data


 Item | Description |
 --- | --- |
 **Objective** | Generate Sales and Logistics Tables |
 **Databricks Run Time** | DBR 16.4 LTS |
 **Language** | Python, Pyspark and SQL |


![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/img/01_diagram_2.png)

## Generate Tables

In [0]:
import pandas as pd
import random
from datetime import date, timedelta

In [0]:
# --- Configuration ---
NUM_PRODUCTS = 500
NUM_STORES = 100
NUM_INVENTORY_RECORDS = 10000
NUM_SALES_RECORDS = 10000

In [0]:
# --- Product Data Generation ---
def generate_product_data(num_products):
    """Generates sample product data."""
    
    product_types = [
        "Funda para Celular", "Auriculares Inalámbricos", "Teclado Mecánico", "Mouse Gamer",
        "Monitor LED", "Cable USB-C", "Cargador Rápido", "Mochila Urbana", "Termo de Acero",
        "Mate de Calabaza", "Bombilla de Alpaca", "Remera de Algodón", "Pantalón de Jean",
        "Zapatillas Deportivas", "Lámpara de Escritorio", "Silla Ergonómica", "Botella Térmica",
        "Parlante Bluetooth", "Smartwatch", "Tablet 10 Pulgadas", "Webcam HD"
    ]
    
    product_descriptors = [
        "Resistente", "Bluetooth 5.0", "RGB", "Ergonómico", "4K Ultra HD",
        "Reforzado", "Inteligente", "Impermeable", "1 Litro", "Premium",
        "Pico de Loro", "Estampada", "Slim Fit", "Para Correr", "Luz Cálida",
        "Soporte Lumbar", "500ml", "Sonido Envolvente", "GPS Integrado",
        "Android 12", "con Micrófono"
    ]
    
    data = []
    product_ids = []
    for i in range(num_products):
        # Cast product_id to string to avoid Arrow conversion errors in Spark
        product_id = str(random.randint(10**18, 10**19 - 1))
        product_ids.append(product_id)
        
        product_name = f"{random.choice(product_types)} {random.choice(product_descriptors)}"
        
        data.append({
            "product_id": product_id,
            "supplier": f"SUPPLIER {random.randint(1, 20)}",
            "product": product_name,
            "upc": str(random.randint(10**12, 10**13 - 1)).zfill(13)
        })
    return pd.DataFrame(data), product_ids

In [0]:
# --- Store Data Generation ---
def generate_store_data(num_stores):
    """Generates sample store data with locations primarily in Argentina."""
    data = []
    store_ids = []
    for i in range(num_stores):
        # Cast store_id to string to avoid Arrow conversion errors in Spark
        store_id = str(random.randint(10**18, 10**19 - 1))
        store_ids.append(store_id)
        
        # Argentina latitude and longitude ranges
        lat = random.uniform(-55.0, -22.0)
        lon = random.uniform(-73.0, -54.0)
        
        data.append({
            "store_id": store_id,
            "store": f"STORE {random.randint(10000, 99999)}",
            "store_type": f"STORE TYPE {random.randint(1, 30)}",
            "store_zip": str(random.randint(1000, 9999)), # Common ZIP format in Argentina
            "store_lat_long": f"{lat:.2f},{lon:.2f}",
            "retailer": f"RETAILER {random.randint(1, 5)}"
        })
    return pd.DataFrame(data), store_ids

In [0]:
# --- Inventory Fact Table Generation ---
def generate_inventory_data(num_records, store_ids, product_ids):
    """Generates sample inventory data, linking stores and products."""
    
    start_date = date(2021, 1, 1)
    end_date = date(2023, 12, 31)
    date_range = (end_date - start_date).days
    
    data = []
    for _ in range(num_records):
        random_days = random.randrange(date_range)
        
        data.append({
            # Cast inventory_id to string to avoid Arrow conversion errors in Spark
            "inventory_id": str(random.randint(10**18, 10**19 - 1)),
            "store_id": random.choice(store_ids),
            "product_id": random.choice(product_ids),
            "date_key": start_date + timedelta(days=random_days),
            "on_hand_quantity": random.randint(0, 100)
        })
    return pd.DataFrame(data)

In [0]:
# --- Sales Fact Table Generation ---
def generate_sales_data(num_records, store_ids, product_ids):
    """Generates sample sales data, linking stores and products."""
    
    start_date = date(2021, 1, 1)
    end_date = date(2023, 12, 31)
    date_range = (end_date - start_date).days
    
    data = []
    for _ in range(num_records):
        random_days = random.randrange(date_range)
        quantity = random.randint(1, 10)
        price = random.uniform(5.0, 500.0)
        
        data.append({
            # Cast sales_id to string to avoid Arrow conversion errors in Spark
            "sales_id": str(random.randint(10**10, 10**11 - 1)),
            "store_id": random.choice(store_ids),
            "product_id": random.choice(product_ids),
            "date_key": start_date + timedelta(days=random_days),
            "sales_quantity": quantity,
            "sales_amount": quantity * price
        })
    return pd.DataFrame(data)

In [0]:
# --- Main execution block ---
if __name__ == "__main__":
    print("Generating dimension tables...")
    # Generate dimension data and get lists of their IDs for joins
    dim_product_pd, product_ids = generate_product_data(NUM_PRODUCTS)
    dim_store_pd, store_ids = generate_store_data(NUM_STORES)

    print("Generating fact tables...")
    # Generate fact data using the dimension IDs
    ft_inventory_pd = generate_inventory_data(NUM_INVENTORY_RECORDS, store_ids, product_ids)
    ft_sales_pd = generate_sales_data(NUM_SALES_RECORDS, store_ids, product_ids)
    
    print("Converting pandas DataFrames to Spark DataFrames...")
    # Convert pandas DataFrames to Spark DataFrames
    # The 'spark' variable is automatically available in Databricks notebooks
    dim_product_df = spark.createDataFrame(dim_product_pd)
    dim_store_df = spark.createDataFrame(dim_store_pd)
    ft_inventory_df = spark.createDataFrame(ft_inventory_pd)
    ft_sales_df = spark.createDataFrame(ft_sales_pd)

    print("Saving DataFrames as tables in Databricks...")
    # Save the DataFrames as tables in Databricks
    # This will overwrite the tables if they already exist
    dim_product_df.write.mode("overwrite").saveAsTable("vinicius_fialho_testes.workshop_ml_agentes.dim_product")
    dim_store_df.write.mode("overwrite").saveAsTable("vinicius_fialho_testes.workshop_ml_agentes.dim_store")
    ft_inventory_df.write.mode("overwrite").saveAsTable("vinicius_fialho_testes.workshop_ml_agentes.ft_inventory")
    ft_sales_df.write.mode("overwrite").saveAsTable("vinicius_fialho_testes.workshop_ml_agentes.ft_sales")
    
    print("\n--- Process Complete ---")
    print(f"Table 'dim_product' created with {dim_product_df.count()} records.")
    print(f"Table 'dim_store' created with {dim_store_df.count()} records.")
    print(f"Table 'ft_inventory' created with {ft_inventory_df.count()} records.")
    print(f"Table 'ft_sales' created with {ft_sales_df.count()} records.")
    print("\nYou can now query these tables using SQL, e.g., 'SELECT * FROM dim_product LIMIT 10'.")

## Display Tables

![](https://raw.githubusercontent.com/Databricks-BR/workshop_agents/refs/heads/main/demo-main/img/img/01_unity_catalog_delta.png)

**Unity Catalog** delivers unified, open governance for data and AI assets by centralizing access management, tracking data lineage, supporting monitoring and auditing, and enhancing data discovery—offering organizations a comprehensive, single source of truth across diverse sources and formats. **Delta Lake**, as a foundational storage layer, ensures performance and reliability through ACID transactions, version control with time travel, and optimized support for both batch and streaming data, enabling interoperability across open formats.

Together, Unity Catalog and Delta Lake offer an integrated solution that combines robust governance with trusted, high-performance storage for modern data environments.

In [0]:
%sql
SELECT *
FROM vinicius_fialho_testes.workshop_ml_agentes.dim_product
LIMIT 10

In [0]:
%sql
SELECT *
FROM vinicius_fialho_testes.workshop_ml_agentes.dim_store
LIMIT 10

In [0]:
%sql
SELECT *
FROM vinicius_fialho_testes.workshop_ml_agentes.ft_inventory
LIMIT 10

In [0]:
%sql
SELECT *
FROM vinicius_fialho_testes.workshop_ml_agentes.ft_sales
LIMIT 10

## Generate Metric Views

### Metric View Inventory

In [0]:
%sql
CREATE OR REPLACE VIEW vinicius_fialho_testes.workshop_ml_agentes.mvw_inventory (
  `Date` COMMENT 'Fecha del registro de inventario (YYYY-MM-DD)',
  `Store ID` COMMENT 'Identificador de la tienda',
  `Store` COMMENT 'Nombre o identificador de la tienda',
  `Store Type` COMMENT 'Tipo o categoría de la tienda',
  `Store Zip` COMMENT 'Código postal de la tienda',
  `Store Lat Long` COMMENT 'Latitud y longitud de la tienda',
  `Retailer` COMMENT 'Nombre del minorista al que pertenece la tienda',
  `Product` COMMENT 'Nombre del producto',
  `Supplier` COMMENT 'Proveedor del producto',
  `Inventory Quantity` COMMENT 'Suma total de la cantidad de inventario disponible'
)
WITH METRICS
LANGUAGE YAML
COMMENT 'Vista Métrica centralizada para el análisis de inventario disponible.'
AS $$

  version: 0.1

  # La tabla de hechos 'ft_inventory' es la fuente principal de nuestros datos numéricos.
  source: vinicius_fialho_testes.workshop_ml_agentes.ft_inventory

  # Unimos las tablas dimensionales para enriquecer los datos de inventario
  # con atributos de tienda y producto.
  joins:
  - name: dim_store
    source: vinicius_fialho_testes.workshop_ml_agentes.dim_store
    using:
    - store_id
  - name: dim_product
    source: vinicius_fialho_testes.workshop_ml_agentes.dim_product
    using:
    - product_id
  
  # Dimensiones: Son los atributos de negocio por los cuales se pueden agrupar o filtrar los datos.
  # Provienen tanto de la tabla de hechos (source) como de las dimensiones unidas.
  dimensions:
  - name: Date
    expr: source.date_key
  - name: Store ID
    expr: dim_store.store_id
  - name: Store
    expr: dim_store.store
  - name: Store Type
    expr: dim_store.store_type
  - name: Store Zip
    expr: dim_store.store_zip
  - name: Store Lat Long
    expr: dim_store.store_lat_long
  - name: Retailer
    expr: dim_store.retailer
  - name: Product
    expr: dim_product.product
  - name: Supplier
    expr: dim_product.supplier

  # Medidas: Son los valores numéricos que se agregan (SUM, AVG, COUNT, etc.).
  measures:
  - name: Inventory Quantity
    expr: SUM(on_hand_quantity)

$$

In [0]:
%sql
SELECT
  `Store`,
  MEASURE(`Inventory Quantity`)
FROM vinicius_fialho_testes.workshop_ml_agentes.mvw_inventory
GROUP BY 1
ORDER BY 2 DESC

### Metric View Sales

In [0]:
%sql
CREATE OR REPLACE VIEW vinicius_fialho_testes.workshop_ml_agentes.mvw_sales (
  `Date` COMMENT 'Fecha de la transacción de venta (YYYY-MM-DD)',
  `Store` COMMENT 'Nombre o identificador de la tienda donde ocurrió la venta',
  `Store Type` COMMENT 'Tipo o categoría de la tienda',
  `Store Zip` COMMENT 'Código postal de la tienda',
  `Retailer` COMMENT 'Nombre del minorista al que pertenece la tienda',
  `Product` COMMENT 'Nombre del producto vendido',
  `Supplier` COMMENT 'Proveedor del producto vendido',
  `Sales Quantity` COMMENT 'Suma total de unidades vendidas',
  `Sales Amount` COMMENT 'Monto total de las ventas en moneda local',
  `Average Sales Ticket` COMMENT 'Métrica clave: Monto promedio por cada unidad vendida (Sales Amount / Sales Quantity).'
)
WITH METRICS
LANGUAGE YAML
COMMENT 'Vista Métrica centralizada para el análisis de rendimiento de ventas.'
AS $$

  version: 0.1

  # La tabla de hechos 'ft_sales' es la fuente principal de nuestros datos numéricos.
  source: vinicius_fialho_testes.workshop_ml_agentes.ft_sales

  # Unimos las tablas dimensionales para enriquecer los datos de ventas
  # con atributos de tienda y producto.
  joins:
  - name: dim_store
    source: vinicius_fialho_testes.workshop_ml_agentes.dim_store
    using:
    - store_id
  - name: dim_product
    source: vinicius_fialho_testes.workshop_ml_agentes.dim_product
    using:
    - product_id
  
  # Dimensiones: Son los atributos de negocio por los cuales se pueden agrupar o filtrar los datos.
  dimensions:
  - name: Date
    expr: source.date_key
  - name: Store
    expr: dim_store.store
  - name: Store Type
    expr: dim_store.store_type
  - name: Store Zip
    expr: dim_store.store_zip
  - name: Retailer
    expr: dim_store.retailer
  - name: Product
    expr: dim_product.product
  - name: Supplier
    expr: dim_product.supplier

  # Medidas: Son los valores numéricos base que se agregan.
  measures:
  - name: Sales Quantity
    expr: SUM(sales_quantity)
  - name: Sales Amount
    expr: SUM(sales_amount)
    
  # --- Métricas Compuestas ---
  # Se recalculan durante la agregación para ser precisas en cualquier nivel de granularidad.
  # Usamos NULLIF para evitar errores de división por cero si la cantidad de ventas es 0.
  - name: Average Sales Ticket
    expr: SUM(sales_amount) / NULLIF(SUM(sales_quantity), 0)

$$

In [0]:
%sql
SELECT
  `Store`,
  MEASURE(`Sales Quantity`)
FROM vinicius_fialho_testes.workshop_ml_agentes.mvw_sales
GROUP BY 1
ORDER BY 2 DESC